In [1]:
from torch.utils.data import DataLoader

from Datasets import BaselineDataset, AllKnobsDataset_train, AllKnobsDataset_test
from Models import TransformerAgent
import torch

In [2]:
# Ensure data directory is populated with https://drive.google.com/drive/folders/1pjHyFlThbpXcvldabLOZZwf4dHaM-p54?usp=share_link

ticker = "AAPL"
data_dir = "../data"
# baseline_dataset = BaselineDataset(ticker, data_dir)
allKnobs_dataset_train = AllKnobsDataset_train(ticker, data_dir)
allKnobs_dataloader_train = DataLoader(allKnobs_dataset_train, batch_size=32, shuffle=True)

allKnobs_dataset_test = AllKnobsDataset_test(ticker, data_dir)
allKnobs_dataloader_test = DataLoader(allKnobs_dataset_test, batch_size=4, shuffle=True)

In [3]:
print(len(allKnobs_dataset_train))
print(len(allKnobs_dataset_test))

1704
365


In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
embedding_dim = len(allKnobs_dataset_train[0][0][0])
hidden_dim = 2048
num_layers = 16
num_atn_heads = 11  # with 22 features, this can only be 1, 2, 11, or 22
checkpoint_dir = "./checkpoints/all_V1"
init_lr = 2
lr_decay = 0.5
min_lr = 0.00000025
decay_lr_every = 10
agent = TransformerAgent(embedding_dim, hidden_dim, num_atn_heads, num_layers, device, checkpoint_dir, init_lr, lr_decay, min_lr, decay_lr_every)

In [ ]:
train_agent = True
if train_agent:
    agent.train(allKnobs_dataloader_train, 500)

Epoch 1, Loss: 556.9349; Average Loss: 0.3268397216785682; lr: [2.0]
Epoch 2, Loss: 445.4446; Average Loss: 0.2614111407821727; lr: [2.0]


In [ ]:
load_agent = False
checkpoint_file = "./checkpoints/all_V1/"

if load_agent:
    agent.load_model(checkpoint_file)

In [ ]:
evaluate_agent = False
if evaluate_agent:
    agent.evaluate(allKnobs_dataloader_test)

In [ ]:
# THERE IS a way to use the loss to measure actual "goodness" since it is scaled in an "absolute" way - 0.02 should mean the same "thing" for one prediction as it does for another

# TODO try L1 loss instead of mse

In [ ]:
# for (X, y) in allKnobs_dataloader:
#     pred_after = torch.ones(32)
#     actual_after = y[:, 0]
#     actual_prev_day = y[:, 1]
#     
#     metric_true = (actual_after - actual_prev_day) / actual_prev_day
#     metric_pred = (pred_after - actual_prev_day) / actual_prev_day
#     print(actual_prev_day.size())
#     break